In [47]:
%reload_ext autoreload

%autoreload 2
import numpy as np
from models import baseline_mlp
import torch
from sklearn.preprocessing import StandardScaler
import random
import pickle

In [48]:
# FILES TO CHANGE (This is where your sequence_nlp_harvey.zip etc is) test

NPY_INPUT_DIR = '/Users/ianmagnusson/IITUDND/data/extracted_features/combined_NLP/maria/kfold/'

OUT_FILE = '/Users/ianmagnusson/IITUDND/notebooks/maria_random_MLP_results.npy'


NUM_EXPERIMENTS = 10000000000000000

In [49]:
def run_experiment(input_dim, hidden_dim, num_layers, activation_function,  learning_rate, momentum, decay_factor, epochs, data_dir):
    
    # load data from files
    X_labeled_train_glove = np.load(data_dir + 'X_labeled_train.npy')
    X_histories_train_glove = np.load(data_dir + 'X_histories_train.npy')
    X_labeled_test_glove = np.load(data_dir + 'X_labeled_test.npy')
    X_histories_test_glove = np.load(data_dir + 'X_histories_test.npy')
    
    y_train = torch.Tensor(np.load(data_dir + 'y_train.npy'))
    y_test = torch.Tensor(np.load(data_dir + 'y_test.npy'))
    
    X_labeled_train_tfidf = np.load(data_dir + 'trainTweets.npy')
    X_histories_train_tfidf = np.load(data_dir + 'trainHistories.npy')
    X_labeled_test_tfidf = np.load(data_dir + 'testTweets.npy')
    X_histories_test_tfidf = np.load(data_dir + 'testHistories.npy')
    
    scaler1 = StandardScaler()
    scaler2 = StandardScaler()
    scaler3 = StandardScaler()
    scaler4 = StandardScaler()
    
    # TODO FIX NAMES!           
    X_labeled_train_glove_norm    = scaler1.fit_transform(X_labeled_train_glove)
    X_histories_train_glove_norm  = scaler2.fit_transform(X_histories_train_glove)
    X_labeled_test_glove_norm     = scaler1.transform(X_labeled_test_glove)
    X_histories_test_glove_norm  = scaler2.transform(X_histories_test_glove)
    
    X_labeled_train_tfidf_norm    = scaler3.fit_transform(X_labeled_train_tfidf)
    X_histories_train_tfidf_norm  = scaler4.fit_transform(X_histories_train_tfidf)
    X_labeled_test_tfidf_norm     = scaler3.transform(X_labeled_test_tfidf)
    X_histories_test_tfidf_norm   = scaler4.transform(X_histories_test_tfidf)
    
    # merge data
    X_train = torch.Tensor(np.concatenate((X_labeled_train_glove_norm, X_labeled_train_tfidf_norm,
                              X_histories_train_glove_norm, X_histories_train_tfidf_norm),
                             axis = 1))
    X_test = torch.Tensor(np.concatenate((X_labeled_test_glove_norm, X_labeled_test_tfidf_norm,
                             X_histories_test_glove_norm, X_histories_test_tfidf_norm),
                            axis = 1))
    
    # build and train model
    model = baseline_mlp.MLP(input_size=input_dim, hidden_dim=hidden_dim, num_layers = num_layers, activation_function = activation_function,
                      learning_rate=learning_rate, momentum = momentum, decay_factor = decay_factor)
    
    model.learn(X_train, y_train, epochs = epochs)
    
    # evaluate
    return model.get_accuracy(X_test, y_test)

def cross_validate(input_dim, hidden_dim, num_layers, activation_function,  learning_rate, momentum, decay_factor, epochs,
                   data_dir, folds = 10):
    accuracy_sum = 0.0
    x = 0
    for i in range(folds):
        fold_dir = data_dir + str(i) + '/'
        print('running fold', i)
        acc = run_experiment(input_dim, hidden_dim, num_layers, activation_function,
                             learning_rate, momentum, decay_factor,epochs,fold_dir)
        print('fold result', acc)
        accuracy_sum += acc
        
    return accuracy_sum / folds


In [50]:
def get_random_params(range_hd, range_nl, range_lr, range_mo, range_dr, range_ep):
    hidden_dim = random.randint(*range_hd)
    num_layers = random.randint(*range_nl)
    activation_function = random.choice([torch.relu, torch.sigmoid, torch.tanh])
    learning_rate = random.uniform(*range_lr)
    momentum = random.uniform(*range_mo)
    decay_factor = random.uniform(*range_dr)
    epochs = random.randint(*range_ep)
    
    return hidden_dim, num_layers, activation_function,  learning_rate, momentum, decay_factor, epochs

In [51]:
range_hd = (50,500)
range_nl = (1, 2)
range_lr = (.1, .0001)
range_mo = (0,1)
range_dr = (.1,1)
range_ep = (1,4)


# run this instead for first time
#results = []

# cross validate search for hyper-parameters
with open(OUT_FILE, 'rb') as fin:
    results = pickle.load(fin)
    
for i in range(NUM_EXPERIMENTS):
    hyperparams = get_random_params(range_hd, range_nl, range_lr, range_mo, range_dr, range_ep)
    print('starting test', i,'params:', hyperparams)
    mean_acc = cross_validate(800, *hyperparams, NPY_INPUT_DIR)
    print('test outcome', mean_acc)
    print('************************************************************')
    results.append((mean_acc, hyperparams))
    results = sorted(results, key=lambda x: x[0], reverse=True)
    print('top so far')
    print(results[:1])
    print('************************************************************')
    
    # backup results
    with open(OUT_FILE, 'wb') as fout:
        pickle.dump(results, fout)

starting test 0 params: (51, 1, <built-in method relu of type object at 0x10cf22540>, 0.014868303212759296, 0.3828602362982968, 0.2737996881789993, 1)
running fold 0
epoch: 0 learning rate: [0.014868303212759296]
[1,   200] loss: 0.631
[1,   400] loss: 0.516
[1,   600] loss: 0.490
fold result 0.8025078369905956
running fold 1
epoch: 0 learning rate: [0.014868303212759296]
[1,   200] loss: 0.607
[1,   400] loss: 0.528
[1,   600] loss: 0.492
fold result 0.7836990595611285
running fold 2
epoch: 0 learning rate: [0.014868303212759296]
[1,   200] loss: 0.642
[1,   400] loss: 0.529
[1,   600] loss: 0.475
fold result 0.7962382445141066
running fold 3
epoch: 0 learning rate: [0.014868303212759296]
[1,   200] loss: 0.621
[1,   400] loss: 0.522
[1,   600] loss: 0.471
fold result 0.7711598746081505
running fold 4
epoch: 0 learning rate: [0.014868303212759296]
[1,   200] loss: 0.616
[1,   400] loss: 0.518
[1,   600] loss: 0.470
fold result 0.8119122257053292
running fold 5
epoch: 0 learning rate: 

In [ ]:
results = sorted(results, key=lambda x: x[0], reverse=True)

print(results[:3])

# RUN THIS CELL AFTER YOU HAVE INTERUPTED OR FINISHED A SEARCH TO UPDATE!!!!!!

with open(OUT_FILE, 'wb') as fout:
    pickle.dump(results, fout)